In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [62]:
# Connect to a local SQLite database. SQLite is a simple file oriented database management "library." 
# in the following we launch a SQLite database stored in the .sqlite file.
dbfile = Path("ZMainBldg.sqlite")
#if dbfile.exists():
    #dbfile.unlink()

sqlite_uri = "sqlite:///ZMainBldg.sqlite"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

In [63]:
# Check tables
sqlite_engine.table_names()

['alameda', 'main_bldg', 'sample']

In [64]:
# Work on sample table for now. sample contains a Simple Random Sample of main_bldg of size 100,000
sql_expr = """
SELECT * FROM sample;
"""
sample = pd.read_sql(sql_expr, sqlite_engine)
sample.head()

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode
0,83062198-B6D5-E511-80C1-3863BB43AC67,14064107,06053,CA,MONTEREY,,092012,20,BKF,701-007-009,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,4-000,,,,,,,,,,,,,,,,,,,,,,,TR 933 THE PINE ACRES 006-372-01 TIME SHARE UNIT 7 WEEK 9,,,,,,,,,402108522,N,,,,N,,,,,,,,,,,0,334415,-1,83062198-B6D5-E511-80C1-3863BB43AC67,,,RESIDENTIAL TIME SHARES,2L,RR000,,,1,,,,,,,,,,,,,,,,,,S,,,,,,,,,,,,,,,,
1,6A3C4527-41D5-E511-80C1-3863BB43AC67,19594047,06083,CA,SANTA BARBARA,2015-07-01,072015,23,BKF,117-414-020,,1,,,,,,515,,,WINDSOR,ST,,515 WINDSOR ST,SANTA MARIA,CA,93458,,,,,,,,2281.6,2014,,,,3-000,,,,,,,,,,,,,,,,,,,,,,,,,0,0.15,6534,,,,,249155007,Y,,,C040,Y,34.9368,-120.442,060830024.021018,,,,,,,,0,316551,-1,6A3C4527-41D5-E511-80C1-3863BB43AC67,0,O,SINGLE FAMILY RESIDENTIAL,0100,RR101,,,1,,,,,,1962,,,,6,4,,2,,0,,,C,0,,,CE,,,,,,1,,,,,,
2,5ABD4B1D-41D5-E511-80C1-3863BB43AC67,10579851,06029,CA,KERN,,082009,17,BKF,004-201-06,,1,,,,,,2701,,,TRUXTUN,AVE,,2701 TRUXTUN AVE,BAKERSFIELD,CA,93301,,,,,,,004201067,1601.03,2009,,,,1-001,,,,,,,,,,,,,,,,,,,,,,,"CITY ABBR CTYBK, BLOCK 415, CITY LOT PTN",,,,,,,,,360377339,Y,,,C001,Y,35.3733,-119.033,060290017.004004,,,,,,,,0,328609,-1,5ABD4B1D-41D5-E511-80C1-3863BB43AC67,,,SINGLE FAMILY RESIDENCE,0101,RR101,,,1,,,,,,1945,1945,,1,7,3,,,,,,1,S,,,,FL,,,,,,,,,,,,
3,F0902D88-BCD5-E511-80C1-3863BB43AC67,17573577,06071,CA,SAN BERNARDINO,,072003,11,BKF,2328-172-05,,1,,,,,,115,,,OLYMPIC,,,115 OLYMPIC,BIG BEAR LAKE,CA,92315,,,,,,,,,,,,,17-001,,,,,,,,,,,,,15,,,BIG BEAR LAKE

In [65]:
# 100000
len(sample)

100000

In [66]:
# Number of sample Alameda records
len(sample[sample['County'] == 'ALAMEDA'])

3854

In [32]:
# Check sample Alameda
sql_expr = """
SELECT * FROM alameda;
"""
alameda_s = pd.read_sql(sql_expr, sqlite_engine)
alameda_s.head()

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode
0,69FBEB4A-2AD5-E511-80C1-3863BB43AC67,9044934,06001,CA,ALAMEDA,,072012,20,BKF,475-147-128,,1,,,,,,33171,,,QUAIL,DR,,33171 QUAIL DR,UNION CITY,CA,94587,,,,,,,,,,,,,15-042,,,,,,,,,,,,,,,,,,,,,,,,,1,0.103,4470,,,,,339869211,Y,,,C034,Y,37.592,-122.042,060014403.073001,,,,,,,,0,332443,-1,69FBEB4A-2AD5-E511-80C1-3863BB43AC67,,,SINGLE FAMILY RESIDENCE,1100,RR101,,,1,,,,,,1979,1981,,1,7,4,,,,,,2,S,,,,,,,,,,,,,,,,
1,9D3017DF-7AD5-E511-80C1-3863BB43AC67,9072079,06001,CA,ALAMEDA,2014-07-01,072014,22,BKF,48H-7590-87,,1,A,,,,P,15,,,STARVIEW,DR,,15 STARVIEW DR,OAKLAND,CA,94618,,,,,,,,6043.92,2014,,,,17-003,,,,,,,,,,,,,,,,,,,,,,,,,1,0.039,1712,,,,ST,9493100,Y,,,C015,Y,37.8525,-122.227,060014001.001060,,,,,,,,0,601,-1,9D3017DF-7AD5-E511-80C1-3863BB43AC67,1,O,PLANNED DEVELOPMENT - TOWNHOUSE,1500,RR104,,,1,,B,,,,1994,1994,,,6,3,,2,,1,,,C,0,,,,,,,,,,,,,,,
2,E474D7B5-8FD5-E511-80C1-3863BB43AC67,9160063,06001,CA,ALAMEDA,,072005,13,BKF,055 -1913-035,,1,,,,,P,2303,,,SPAULDING,AVE,,2303 SPAULDING AVE,BERKELEY,CA,94703,,,,,,,,,,,,,13-000,,,,,,,,,,,,,,,,,,,,,,,,,1,,4158,,,,,336628977,Y,,,C009,Y,37.8658,-122.281,060014230.003005,,,,,,,,0,334117,-1,E474D7B5-8FD5-E511-80C1-3863BB43AC67,,O,SINGLE FAMILY RESIDENCE,1100,RR101,,,1,,,,,,1925,,,,7,3,,,,,,1.5,S,,,,,,,,,,,,,,,,
3,0EDE9B49-2AD5-E511-80C1-3863BB43AC67,8923761,06001,CA,ALAMEDA,,072012,20,BKF,15-1364-17,,1,,,,,,666,,,63RD,ST,,666 63RD ST,OAKLAND,CA,94609,,,,,,,,,,,,,17-003,,,,,,,,,,,,,,,,,,,,,,,,,1,0.14,6100,,,,,340076500,Y,,,C00

In [59]:
# Get cities from JC RC Inventory
cities = list(pd.read_excel('jc_rc.xlsx')['Unnamed: 0'].values[1:len(counties)-1])

In [67]:
# CREATE TABLE alameda AS SELECT * FROM main_bldg WHERE County='ALAMEDA';
sqlite_uri = "sqlite:///ZCountyExtracts.sqlite"
zce_engine = sqlalchemy.create_engine(sqlite_uri)

In [70]:
zce_engine.table_names()

['alameda']

In [71]:
sql_expr = """
SELECT * FROM alameda;
"""
alameda = pd.read_sql(sql_expr, zce_engine)
alameda.head()

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode
0,083BFB02-2AD5-E511-80C1-3863BB43AC67,8904539,06001,CA,ALAMEDA,,082010,18,BKF,1-1111-10,,1,,,,,P,311,,,OAK,ST,,311 OAK ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,,,,,,339056319,Y,APT,105,C010,Y,37.7935,-122.268,060014033.001023,,,,,,,,0,330739,-1,083BFB02-2AD5-E511-80C1-3863BB43AC67,,O,CONDOMINIUM,7301,RR106,,,1,,,,,,2003,2003,,10,3,1,,,,,,2,S,,,,,,,Y,,,,,,,,,
1,093BFB02-2AD5-E511-80C1-3863BB43AC67,8904540,06001,CA,ALAMEDA,,082010,18,BKF,1-1111-11,,1,,,,,P,311,,,OAK,ST,,311 OAK ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,,,,,,339056320,Y,APT,106,C010,Y,37.7935,-122.268,060014033.001023,,,,,,,,0,330739,-1,093BFB02-2AD5-E511-80C1-3863BB43AC67,,O,CONDOMINIUM,7301,RR106,,,1,,,,,,2003,2003,,10,3,1,,,,,,2,S,,,,,,,Y,,,,,,,,,
2,0A3BFB02-2AD5-E511-80C1-3863BB43AC67,8904541,06001,CA,ALAMEDA,,082010,18,BKF,1-1111-12,,1,,,,,,311,,,OAK,ST,,311 OAK ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,,,,,,339056321,Y,APT,107,C010,Y,37.7935,-122.268,060014033.001023,,,,,,,,0,330739,-1,0A3BFB02-2AD5-E511-80C1-3863BB43AC67,,,CONDOMINIUM,7301,RR106,,,1,,,,,,2003,2003,,10,3,1,,,,,,2,S,,,,,,,Y,,,,,,,,,
3,0B3BFB02-2AD5-E511-80C1-3863BB43AC67,8904542,06001,CA,ALAMEDA,,082010,18,BKF,1-1111-13,,1,,,,,,300,,,OAK,ST,,300 OAK ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,,,,,,339056322,Y,#,108,,Y,37.7934,-122.267,060014033.001024,,,,,,,,0,330739,-1,0B3BFB02-2AD5-E511-80C1-3863BB43AC67,,,CONDOMINIUM,7301,RR

In [73]:
# Number of Alameda records
len(alameda)

5286873